## Criando a camada gold


In [0]:
%sql
use catalog medalhao;
create schema if not exists gold;

## 1º Projeto — Área de Logística (Vendas por
## Localidade)

### 1.1 Criação da tabela gold.ft_vendas_consumidor_local

In [0]:
from pyspark.sql import functions as F

In [0]:
ft_pedidos_total_silver = spark.table("silver.ft_pedido_total")
ft_consumidores_silver = spark.table("silver.ft_consumidores")


df_venda_consumidor_gold = (
    ft_pedidos_total_silver.alias("t").join(
    ft_consumidores_silver.alias("c") , on='id_consumidor', how='left')
     .select(
        F.col("t.id_pedido").cast("string"),
        F.col("t.id_consumidor").cast("string"),
        F.col("t.valor_total_pago_brl").alias("valor_total_pedido_brl").cast("decimal(12,2)"),
        F.col("c.cidade").cast("string"),
        F.col("c.estado").cast("string"),
        F.col("t.data_pedido").cast("date")
    )
)

df_venda_consumidor_gold.write.mode("overwrite").format("delta").saveAsTable("gold.ft_vendas_consumidor_local")


In [0]:
spark.table("gold.ft_vendas_consumidor_local").show(10)


### 1.2 Criação da view
### gold.view_total_compras_por_consumidor



In [0]:
%sql
CREATE OR REPLACE VIEW gold.view_total_compras_por_consumidor AS
SELECT
    cidade,
    estado,
    COUNT(*) AS quantidade_vendas,
    SUM(valor_total_pedido_brl) AS valor_total_localidade
FROM gold.ft_vendas_consumidor_local
GROUP BY cidade, estado;


Criando consulta para vizualizar total de vendas por estado a  partir da view criada

In [0]:
%sql
use catalog medalhao;
SELECT
    estado,
    SUM(quantidade_vendas) AS total_vendas_estado,
    SUM(valor_total_localidade) AS total_valor_vendido_estado
FROM gold.view_total_compras_por_consumidor
GROUP BY estado
ORDER BY total_valor_vendido_estado DESC;

## 2º Projeto — Área de Logística (Análise de Atrasos
## de Entregas)

### 2.1 Criação da tabela
### gold.ft_atrasos_pedidos_local_vendedor:


In [0]:
ft_pedidos_silver = spark.table("silver.ft_pedidos")
ft_consumidores_silver = spark.table("silver.ft_consumidores")
ft_itens_pedidos_silver = spark.table("silver.ft_itens_pedidos")

df_inicial = (
    ft_pedidos_silver.alias("p")
    .join(ft_consumidores_silver.alias("c"), on="id_consumidor", how="left")
    .join(ft_itens_pedidos_silver.alias("i"), on="id_pedido", how="left") 
)

df_atrasos_pedidos_gold = df_inicial.select(
    F.col("p.id_pedido").cast("string"),
    F.col("i.id_vendedor").cast("string"),
    F.col("p.id_consumidor").cast("string"),
    F.col("p.entrega_no_prazo").cast("string"),
    F.col("p.tempo_entrega_dias").cast("integer"),
    F.col("p.tempo_entrega_estimado_dias").cast("integer"),
    F.col("c.cidade").cast("string"),
    F.col("c.estado").cast("string")
)

In [0]:
df_atrasos_pedidos_gold.write.mode("overwrite").format("delta").saveAsTable("gold.ft_atrasos_pedidos_local_vendedor")


In [0]:
# O primeiro foi .show mas nesee ele ficou meio desorganizado no meu entao usei display
display(df_atrasos_pedidos_gold)

### 2.2 Criação das Views Analíticas


2.2.1 gold.view_tempo_medio_entrega_localidade

In [0]:
%sql
CREATE OR REPLACE VIEW gold.view_tempo_medio_entrega_localidade AS
WITH medias AS (
    SELECT
        cidade,
        estado,
        AVG(tempo_entrega_dias) AS tempo_medio_entrega,
        AVG(tempo_entrega_estimado_dias) AS tempo_medio_estimado
    FROM gold.ft_atrasos_pedidos_local_vendedor
    GROUP BY cidade, estado
)

SELECT
    cidade,
    estado,
    tempo_medio_entrega,
    tempo_medio_estimado,
    CASE
        WHEN tempo_medio_entrega IS NULL THEN 'NÃO' -- Considerando os casos em que o pedido nao foi entregue
        WHEN tempo_medio_entrega > tempo_medio_estimado THEN 'SIM'
        ELSE 'NÃO'
    END AS entrega_maior_que_estimado
FROM medias;


Vizualizando view

In [0]:
%sql
SELECT *
FROM gold.view_tempo_medio_entrega_localidade
ORDER BY tempo_medio_entrega DESC;


2.2.2 gold.view_vendedor_pontualidade


In [0]:
%sql
CREATE OR REPLACE VIEW gold.view_vendedor_pontualidade AS
SELECT
    id_vendedor,
    COUNT(*) AS total_pedidos,
    SUM(CASE WHEN entrega_no_prazo IN ('Não', 'Não Entregue') THEN 1 ELSE 0 END) AS total_atrasados,
    ROUND(
        SUM(CASE WHEN entrega_no_prazo IN ('Não', 'Não Entregue') THEN 1 ELSE 0 END) * 100.0 
        / COUNT(*),
        2
    ) AS percentual_atraso
FROM gold.ft_atrasos_pedidos_local_vendedor
GROUP BY id_vendedor; 

Vizualizando view

In [0]:
%sql
SELECT *
FROM gold.view_vendedor_pontualidade
ORDER BY percentual_atraso DESC

## 3º Projeto — Área Comercial (Análises de Vendas
## por Período)

### 3.1 Criação da Dimensão de Tempo — gold.dm_tempo


In [0]:
# Para criar essa dimensao de forma mais eficiente podemos pegar o intervalo de tempo do primeiro e ultimo pedido feito
df_pedidos_silver = spark.table("silver.ft_pedidos")

df_min_max = df_pedidos_silver.select(
    F.min("pedido_compra_timestamp").alias("data_inicio"),
    F.max("pedido_compra_timestamp").alias("data_fim")
)

# Gerando as datas no intervalo de tempo encontrado usando sequence e explode 
df_tempo = (
    df_min_max.select(
        F.explode(
            F.sequence(
                F.to_date("data_inicio"),
                F.to_date("data_fim"),
                F.expr("INTERVAL 1 DAY")
            )
        ).alias("sk_tempo")
    )
)



In [0]:
df_tempo = (
    df_tempo
        .withColumn("ano", F.year("sk_tempo"))
        .withColumn("trimestre", F.quarter("sk_tempo"))
        .withColumn("mes", F.month("sk_tempo"))
        .withColumn("semana_do_ano", F.weekofyear("sk_tempo"))
        .withColumn("dia", F.dayofmonth("sk_tempo"))
        .withColumn("dia_da_semana_num", F.dayofweek("sk_tempo"))
        
        .withColumn(
            "dia_da_semana_nome",
            F.when(F.col("dia_da_semana_num") == 1, "Domingo")
             .when(F.col("dia_da_semana_num") == 2, "Segunda-feira")
             .when(F.col("dia_da_semana_num") == 3, "Terça-feira")
             .when(F.col("dia_da_semana_num") == 4, "Quarta-feira")
             .when(F.col("dia_da_semana_num") == 5, "Quinta-feira")
             .when(F.col("dia_da_semana_num") == 6, "Sexta-feira")
             .when(F.col("dia_da_semana_num") == 7, "Sábado")
        )
        
        .withColumn(
            "mes_nome",
            F.when(F.col("mes") == 1,  "Janeiro")
             .when(F.col("mes") == 2,  "Fevereiro")
             .when(F.col("mes") == 3,  "Março")
             .when(F.col("mes") == 4,  "Abril")
             .when(F.col("mes") == 5,  "Maio")
             .when(F.col("mes") == 6,  "Junho")
             .when(F.col("mes") == 7,  "Julho")
             .when(F.col("mes") == 8,  "Agosto")
             .when(F.col("mes") == 9,  "Setembro")
             .when(F.col("mes") == 10, "Outubro")
             .when(F.col("mes") == 11, "Novembro")
             .when(F.col("mes") == 12, "Dezembro")
        )
        
        # Indicador de final de semana
        .withColumn(
            "eh_fim_de_semana",
            F.when(F.col("dia_da_semana_num").isin(1,7), "Sim")
             .otherwise("Não")
        )
)


In [0]:
df_tempo.write.mode("overwrite").format("delta").saveAsTable("gold.dm_tempo")


In [0]:
display(df_tempo)
# Colocar nome do dia e mes em portugues

### 3.2 Criação da Fato gold.ft_vendas_geral


In [0]:
pedidos = spark.table("silver.ft_pedidos")
itens = spark.table("silver.ft_itens_pedidos")
cotacao = spark.table("silver.dm_cotacao_dolar")
avaliacoes = spark.table("silver.ft_avaliacoes_pedidos")

avaliacao_media = (
    avaliacoes.groupBy("id_pedido")
              .agg(F.avg("avaliacao").alias("avaliacao_pedido"))
)

df_geral = (
    pedidos.alias("p")
    .join(itens.alias("i"), F.col("p.id_pedido") == F.col("i.id_pedido"))
    .join(avaliacao_media.alias("a"), "id_pedido", "left")
    .join(
        cotacao.alias("cot"),
        F.to_date("p.pedido_compra_timestamp") == F.col("cot.data"),
        "left"
    )
)

df_vendas_geral_gold = (
    df_geral.select(
        F.col("p.id_pedido"),
        F.col("id_item"),

        # FKs
        F.col("id_consumidor").alias("fk_cliente"),
        F.col("id_produto").alias("fk_produto"),
        F.col("id_vendedor").alias("fk_vendedor"),
        F.to_date("p.pedido_compra_timestamp").alias("fk_tempo"),

        F.col("status").alias("status_pedido"),

        F.col("p.tempo_entrega_dias"),
        F.col("p.entrega_no_prazo"),

        # Valores BRL
        F.col("i.preco_BRL").cast("DECIMAL(12,2)").alias("valor_produto_brl"),
        F.col("i.preco_frete").cast("DECIMAL(12,2)").alias("valor_frete_brl"),
        (F.col("i.preco_BRL") + F.col("i.preco_frete")).alias("valor_total_item_brl"),
        
        # Conversão USD
        F.round(F.col("i.preco_BRL") / F.col("cot.cotacao_dolar"), 2).cast("DECIMAL(12,2)").alias("valor_produto_usd"),
        F.round(F.col("i.preco_frete") / F.col("cot.cotacao_dolar"), 2).cast("DECIMAL(12,2)").alias("valor_frete_usd"),
        F.round(
            (F.col("i.preco_BRL") + F.col("i.preco_frete")) / F.col("cot.cotacao_dolar"),
            2
        ).alias("valor_total_item_usd"),

       
        F.col("cot.cotacao_dolar").cast("DECIMAL(8,4)").alias("cotacao_dolar"),
        F.col("a.avaliacao_pedido").cast("DECIMAL(3,2)").alias("avaliacao_pedido")
    )
)


In [0]:
df_vendas_geral_gold.write.format("delta").mode("overwrite").saveAsTable("gold.ft_vendas_geral")

In [0]:
spark.table("gold.ft_vendas_geral").display()


### 3.3 Criação da view gold.view_vendas_por_periodo

In [0]:
%sql
CREATE OR REPLACE VIEW gold.view_vendas_por_periodo AS
SELECT
    t.ano,
    t.trimestre,
    t.mes,
    t.mes_nome,
    t.dia,
    t.dia_da_semana_num,

    -- Métricas
    COUNT(DISTINCT f.id_pedido) AS total_pedidos,
    COUNT(f.id_item) AS total_itens,

    SUM(f.valor_total_item_brl) AS receita_total_brl,
    SUM(f.valor_total_item_usd) AS receita_total_usd,

    ROUND(AVG(f.valor_total_item_brl), 2) AS ticket_medio_brl,

    ROUND(AVG(f.avaliacao_pedido), 2) AS avaliacao_media

FROM gold.ft_vendas_geral f
JOIN gold.dm_tempo t
    ON f.fk_tempo = t.sk_tempo

GROUP BY
    t.ano,
    t.trimestre,
    t.mes,
    t.mes_nome,
    t.dia,
    t.dia_da_semana_num

ORDER BY
    t.ano,
    t.mes,
    t.dia;


In [0]:
%sql
SELECT *
FROM gold.view_vendas_por_periodo

3.3.1 Consultas Analíticas


1. Qual é o dia da semana com maior receita total em reais ( receita_total_brl )?


In [0]:
%sql
SELECT dia_da_semana_num, SUM(receita_total_brl) AS receita_total
FROM gold.view_vendas_por_periodo
GROUP BY dia_da_semana_num
ORDER BY receita_total DESC

2. Considerando o último ano disponível na dimensão de tempo, qual foi o
mês com maior ticket médio ( ticket_medio_brl )?


In [0]:
%sql
WITH ultimo_ano AS (
    SELECT MAX(ano) AS ano_max
    FROM gold.view_vendas_por_periodo
)
SELECT v.ano, v.mes, v.mes_nome, AVG(v.ticket_medio_brl) AS ticket_medio_mes
FROM gold.view_vendas_por_periodo v
JOIN ultimo_ano u ON v.ano = u.ano_max
GROUP BY v.ano, v.mes, v.mes_nome
ORDER BY ticket_medio_mes DESC

### 3.4. Criação da gold.view_top_produto


In [0]:
%sql
CREATE OR REPLACE VIEW gold.view_top_produto AS
SELECT
    p.id_produto,
    p.categoria_produto AS categoria_produto,
    
    COUNT(g.id_item) AS quantidade_vendida,
    COUNT(DISTINCT g.id_pedido) AS total_pedidos,
    
    CAST(SUM(g.valor_total_item_brl) AS DECIMAL(12,2)) AS receita_brl,
    CAST(SUM(g.valor_total_item_usd) AS DECIMAL(12,2)) AS receita_usd,

    CAST(AVG(g.valor_produto_brl) AS DECIMAL(12,2)) AS preco_medio_brl,
    CAST(AVG(g.avaliacao_pedido) AS DECIMAL(3,2)) AS avaliacao_media,

    CAST(AVG(p.peso_produto_gramas) AS DECIMAL(8,2)) AS peso_medio_gramas

FROM gold.ft_vendas_geral g
LEFT JOIN silver.ft_produtos p
    ON g.fk_produto = p.id_produto

GROUP BY 
    p.id_produto,
    p.categoria_produto;


In [0]:
%sql
SELECT * FROM gold.view_top_produto

### 3.5 Criação da view_vendas_produtos_esteticos


In [0]:
%sql
use catalog medalhao;
CREATE OR REPLACE VIEW gold.view_vendas_produtos_esteticos AS
WITH itens_informacoes AS (
    SELECT
        i.id_pedido,
        i.id_item,
        i.id_produto,
        p.categoria_produto,
        t.ano,
        t.mes,
        i.preco_BRL,
        
        CASE
            WHEN cot.cotacao_dolar IS NULL OR cot.cotacao_dolar = 0 THEN NULL
            ELSE ROUND(i.preco_BRL / cot.cotacao_dolar, 2)
        END AS preco_USD_item
    FROM silver.ft_itens_pedidos i
    INNER JOIN silver.ft_pedido_total total
        ON i.id_pedido = total.id_pedido
    INNER JOIN silver.ft_produtos p
        ON i.id_produto = p.id_produto
    INNER JOIN gold.dm_tempo t
        ON total.data_pedido = t.sk_tempo
    LEFT JOIN silver.dm_cotacao_dolar cot
        ON total.data_pedido = cot.data
    WHERE p.categoria_produto LIKE 'fashion%'
),

order_categories AS (
    SELECT DISTINCT
        i.id_pedido,
        i.categoria_produto,
        i.ano,
        i.mes
    FROM itens_informacoes i
),

avaliacoes_por_categoria AS (
    SELECT
        oc.ano,
        oc.mes,
        oc.categoria_produto,
        CAST(AVG(a.avaliacao) AS DECIMAL(3,2)) AS avaliacao_media
    FROM order_categories oc
    LEFT JOIN silver.ft_avaliacoes_pedidos a
        ON oc.id_pedido = a.id_pedido
    GROUP BY oc.ano, oc.mes, oc.categoria_produto
),

metrificacao AS (
    SELECT
        i.ano,
        i.mes,
        i.categoria_produto,
        COUNT(DISTINCT i.id_pedido) AS total_pedidos,
        COUNT(*) AS total_itens_vendidos,
        CAST(SUM(i.preco_BRL) AS DECIMAL(12,2)) AS receita_total_brl,
        CAST(SUM(i.preco_USD_item) AS DECIMAL(12,2)) AS receita_total_usd,
        CAST(AVG(i.preco_BRL) AS DECIMAL(12,2)) AS ticket_medio_brl,
        CAST(AVG(i.preco_USD_item) AS DECIMAL(12,2)) AS ticket_medio_usd
    FROM itens_informacoes i
    GROUP BY i.ano, i.mes, i.categoria_produto
)

SELECT
    m.ano,
    m.mes,
    m.categoria_produto,
    m.total_pedidos,
    m.total_itens_vendidos,
    m.receita_total_brl,
    m.receita_total_usd,
    m.ticket_medio_brl,
    m.ticket_medio_usd,
    COALESCE(a.avaliacao_media, CAST(NULL AS DECIMAL(3,2))) AS avaliacao_media
FROM metrificacao m
LEFT JOIN avaliacoes_por_categoria a
    ON m.ano = a.ano
   AND m.mes = a.mes
   AND m.categoria_produto = a.categoria_produto
ORDER BY m.ano, m.mes, m.categoria_produto;


In [0]:
%sql
SELECT *
FROM gold.view_vendas_produtos_esteticos